# 1. Extract
## Extracción de los datos que están en el contenedor del Storage Account de Microsoft Azure.

## Definiendo variables necesarias para ingresar al contenedor

In [ ]:
storage_account_name = "storage_account"
storage_account_access_key = "account_acces_key"
container = "datos"
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",storage_account_access_key)

# Listamos el path para cada archivo junto con su nombre y demás propiedades
files_raw = dbutils.fs.ls(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/")
# A partir de la lista de dbutils realizamos una nueva lista donde obtenemos las URLs para cada archivo
file_list = [str(i).split('\'')[1] for i in files_raw]
# Creamos un path específico para metadata
metadata_path = str(dbutils.fs.ls(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/metadata.json")[0]).split('\'')[1]

### Creando los dataframes: Definimos una función que reciba el path del archivo y nos genere el dataframe respectivo

In [ ]:
def get_df(file_location):
    file_type = "json"
    spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)
    df = spark.read.format(file_type).option("inferSchema", "true").load(file_location)
    return df

In [ ]:
# Creating dataframes
amazon_instant_video = get_df(file_list[0])
apps_for_android = get_df(file_list[1])
automotive = get_df(file_list[2])
baby = get_df(file_list[3])
beauty = get_df(file_list[4])
books = get_df(file_list[5])
cds_and_vinyl = get_df(file_list[6])
cell_phones_accessories = get_df(file_list[7])
clothing_shoes_jewelry = get_df(file_list[8])
digital_music = get_df(file_list[9])  
electronics = get_df(file_list[10])
grocery_and_gourmet_food = get_df(file_list[11]) 
health_and_personal = get_df(file_list[12])
home_and_kitchen = get_df(file_list[13])
kindle_store = get_df(file_list[14])  
movies_and_tv = get_df(file_list[15])
musical_instruments = get_df(file_list[16])
office_products = get_df(file_list[17])
patio_lawn_garden = get_df(file_list[18])
pet_supplies = get_df(file_list[19])
sports_and_outdoors = get_df(file_list[20])
Tools_and_Home_Improvement = get_df(file_list[21])
toys_and_games = get_df(file_list[22])
videogames = get_df(file_list[23]) 

In [ ]:
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, ArrayType, LongType
from pyspark.sql.functions import col, from_json
# To handle corrupt records in metadata we specify a schema
schema = StructType([
	StructField('asin',StringType(),True),
	StructField('brand',StringType(),True),
	StructField('categories',StringType(),True),
	StructField('description',StringType(),True),
	StructField('imUrl',StringType(),True),
	StructField('price',DoubleType(),True),
	StructField('related',StructType([
		StructField('also_bought',ArrayType(StringType(),True),True),
		StructField('also_viewed',ArrayType(StringType(),True),True),
		StructField('bought_together',ArrayType(StringType(),True),True),
		StructField('buy_after_viewing',ArrayType(StringType(),True),True)]),True),
	StructField('salesRank',StructType([
		StructField('Appliances',LongType(),True),
		StructField('Arts, Crafts & Sewing',LongType(),True),
		StructField('Automotive',LongType(),True),
		StructField('Baby',LongType(),True),
		StructField('Beauty',LongType(),True),
		StructField('Books',LongType(),True),
		StructField('Camera &amp; Photo',LongType(),True),
		StructField('Cell Phones & Accessories',LongType(),True),
		StructField('Clothing',LongType(),True),
		StructField('Computers & Accessories',LongType(),True),
		StructField('Electronics',LongType(),True),
		StructField('Gift Cards Store',LongType(),True),
		StructField('Grocery & Gourmet Food',LongType(),True),
		StructField('Health & Personal Care',LongType(),True),
		StructField('Home &amp; Kitchen',LongType(),True),
		StructField('Home Improvement',LongType(),True),
		StructField('Industrial & Scientific',LongType(),True),
		StructField('Jewelry',LongType(),True),
		StructField('Kitchen & Dining',LongType(),True),
		StructField('Magazines',LongType(),True),
		StructField('Movies & TV',LongType(),True),
		StructField('Music',LongType(),True),
		StructField('Musical Instruments',LongType(),True),
		StructField('Office Products',LongType(),True),
		StructField('Patio, Lawn & Garden',LongType(),True),
		StructField('Pet Supplies',LongType(),True),
		StructField('Prime Pantry',LongType(),True),
		StructField('Shoes',LongType(),True),
		StructField('Software',LongType(),True),
		StructField('Sports &amp; Outdoors',LongType(),True),
		StructField('Toys & Games',LongType(),True),
		StructField('Video Games',LongType(),True),
		StructField('Watches',LongType(),True)]),True),
	StructField('title',StringType(),True),
	StructField('_corrupt_record',StringType(),True)
])

In [ ]:
# Read JSON data into a spark dataframe using our schema
metadata_df = spark.read.option('mode', 'PERMISSIVE').schema(schema).option('header', True).option('columnNameOfCorruptRecord', '_corrupt_record').json(metadata_path).cache()
# Separar las filas corruptas en un dataframe nuevo
corrupt_df = metadata_df.filter(col('_corrupt_record').isNotNull())

# 2. Transform

## Funciones

In [ ]:
def unionAll(*dfs):
    df_union = reduce(DataFrame.unionAll, dfs)
    return df_union

def set_nullable_false(spark, df, columns, nullable = False):
    for struct_field in df.schema:
        if struct_field.name in columns:
            struct_field.nullable = nullable
    df_mod = spark.createDataFrame(df.rdd, df.schema)
    return df_mod

## Trabajando con la tabla REVIEWS

In [ ]:
# Uniendo los dataframes
from pyspark.sql import DataFrame
from functools import reduce

df_complete = unionAll(amazon_instant_video, apps_for_android, automotive, baby, beauty, books, cds_and_vinyl, cell_phones_accessories, clothing_shoes_jewelry, digital_music, electronics, grocery_and_gourmet_food, health_and_personal, home_and_kitchen, kindle_store, movies_and_tv, musical_instruments, office_products,patio_lawn_garden, pet_supplies, sports_and_outdoors, Tools_and_Home_Improvement, toys_and_games, videogames)

In [ ]:
df_complete.printSchema()

root
-- asin: string (nullable = true)
-- helpful: array (nullable = true)
 |-- element: long (containsNull = true)
-- overall: double (nullable = true)
-- reviewText: string (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- reviewerName: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: long (nullable = true)

In [ ]:
# Renombrando la columna ASIN y eliminando la columna de reviewTime
df_complete = df_complete.withColumnRenamed("asin", "asinID")
df_complete = df_complete.drop('reviewTime')

In [ ]:
# Trabajando sobre la columna Helpful -> Creando dos nuevas columnas que separen los datos
from pyspark.sql.functions import expr
df_complete = df_complete.select(["asinID", "reviewText", "reviewerID", "reviewerName", "summary", "unixReviewTime", "overall", "helpful"]+[expr("helpful[" + str(x)+ "]") for x in range(0, 2)]).drop('helpful')

In [ ]:
# Cambiando UnixTime a Datetime para mejorar el filtrado
from pyspark.sql.functions import from_unixtime
df_complete = df_complete.withColumn("datetime", from_unixtime("unixReviewTime")).drop("UnixReviewTime")

In [ ]:
# Agregando la columna reviewID
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.window import Window
df_complete = df_complete.withColumn("reviewID", monotonically_increasing_id())
df_complete = df_complete.select(['reviewID', 'reviewerID', 'asinID', 'reviewerName', 'reviewText', 'summary', 'helpful[0]', 'helpful[1]', 'overall', 'datetime'])

In [ ]:
reviews = set_nullable_false(spark, df_complete, ["reviewID", "reviewerID", "asinID"])
from pyspark.sql.functions import to_date
reviews = reviews.select("*", to_date("datetime").alias("date")).drop("datetime")

In [ ]:
reviews.printSchema()

root
-- reviewID: long (nullable = false)
-- reviewerID: string (nullable = false)
-- asinID: string (nullable = false)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- helpful[0]: long (nullable = true)
-- helpful[1]: long (nullable = true)
-- overall: double (nullable = true)
-- date: date (nullable = true)

In [ ]:
# Agregando la tabla a la base de datos SQL de databricks
reviews.write.format("delta").saveAsTable("reviews")

#### Definición de querys para carga de datos delta

In [ ]:
# Reviewer Table
df_reviewerid_name = spark.sql("SELECT DISTINCT(reviewerID), reviewerName FROM reviews WHERE YEAR(date) BETWEEN 2000 AND 2014")
df_reviewerid_name = df_reviewerid_name.dropDuplicates()

In [ ]:
# Average Product Score
df_average_product = spark.sql("SELECT asinID, AVG(overall) AS average_Overall FROM reviews WHERE YEAR(date) BETWEEN 2000 AND 2014 GROUP BY asinID")

## Trabajando con la tabla metadata

In [ ]:
# Empezamos a limpiar las filas corruptas
# Primero definimos un nuevo schema que coincida con el de las filas corruptas
schema = StructType([
	StructField('asin',StringType(),True),
	StructField('categories',StringType(),True),
	StructField('description',StringType(),True),
    StructField('title',StringType(),True),
    StructField('price',DoubleType(),True),
    StructField('salesRank',StructType([
		StructField('Appliances',LongType(),True),
		StructField('Arts, Crafts & Sewing',LongType(),True),
		StructField('Automotive',LongType(),True),
		StructField('Baby',LongType(),True),
		StructField('Beauty',LongType(),True),
		StructField('Books',LongType(),True),
		StructField('Camera &amp; Photo',LongType(),True),
		StructField('Cell Phones & Accessories',LongType(),True),
		StructField('Clothing',LongType(),True),
		StructField('Computers & Accessories',LongType(),True),
		StructField('Electronics',LongType(),True),
		StructField('Gift Cards Store',LongType(),True),
		StructField('Grocery & Gourmet Food',LongType(),True),
		StructField('Health & Personal Care',LongType(),True),
		StructField('Home &amp; Kitchen',LongType(),True),
		StructField('Home Improvement',LongType(),True),
		StructField('Industrial & Scientific',LongType(),True),
		StructField('Jewelry',LongType(),True),
		StructField('Kitchen & Dining',LongType(),True),
		StructField('Magazines',LongType(),True),
		StructField('Movies & TV',LongType(),True),
		StructField('Music',LongType(),True),
		StructField('Musical Instruments',LongType(),True),
		StructField('Office Products',LongType(),True),
		StructField('Patio, Lawn & Garden',LongType(),True),
		StructField('Pet Supplies',LongType(),True),
		StructField('Prime Pantry',LongType(),True),
		StructField('Shoes',LongType(),True),
		StructField('Software',LongType(),True),
		StructField('Sports &amp; Outdoors',LongType(),True),
		StructField('Toys & Games',LongType(),True),
		StructField('Video Games',LongType(),True),
		StructField('Watches',LongType(),True)]),True),
	StructField('imUrl',StringType(),True),
	StructField('related',StructType([
		StructField('also_bought',ArrayType(StringType(),True),True),
		StructField('also_viewed',ArrayType(StringType(),True),True),
		StructField('bought_together',ArrayType(StringType(),True),True),
		StructField('buy_after_viewing',ArrayType(StringType(),True),True)]),True)
])

clean_df = corrupt_df.select(from_json(col('_corrupt_record'), schema).alias('data')).select('data.*')
print(f"Filas limpiadas: {clean_df.dropna('all').count()}") # 1026369

Filas limpiadas: 1026369

In [ ]:
from pyspark.sql.functions import lit
# Agregar columnas faltantes a df limpio para unirlo al df principal
clean_df = clean_df.withColumn('brand', lit(None)).select('asin','brand','categories','description','imUrl','price','related','salesRank','title')
# Quitamos filas corruptas del dataframe de metadata
metadata_df = metadata_df.where(col('_corrupt_record').isNull()).drop('_corrupt_record')
dfcount = metadata_df.count()
print(f"Cantidad de filas de metadata sin datos corruptos: {dfcount}")
# Agregar filas limpias a df metadata
metadata_df = metadata_df.union(clean_df.dropna('all'))
dfcount = metadata_df.count()
print(f"Metadata con algunas filas limpias agregadas: {dfcount}")
print(f"Quedaron por limpiar {9430088 - dfcount} filas.")

Cantidad de filas de metadata sin datos corruptos: 8403388
Metadata con algunas filas limpias agregadas: 9429757
Quedaron por limpiar 331 filas.

#### Quedaron 331 (0.0035%) filas corruptas que tienen un schema diferente y no pudieron ser limpiadas.

#### Cantidad de valores nulos en cada columna (antes y después de limpieza):

#### Columnas ordenadas por datos faltantes:
brand: 7.851.347<br>
description: 3.728.744<br>
price: 3.366.714<br>
related: 2.738.259<br>
salesRank: 2.453.213<br>
title: 1.432.617<br>
imUrl: 169.298<br>
categories: 75.249<br>
asin: 0<br>

In [ ]:
# Insertar dataframe metadata en tabla
metadata_df.write.insertInto('products5')

In [ ]:
# Insertar datos corruptos en una tabla separada
corrupt_df.write.insertInto('corrupt_products')

#### Cleaning corrupt data

In [ ]:
%sql
UPDATE corrupt_products SET asin = regexp_extract(_corrupt_record, "(?<=asin': ')(.*?)(?=', 'title|', 'description|', 'price|', 'salesRank|', 'imUrl|', 'related|', 'brand|', 'categories|'})", 1);
UPDATE corrupt_products SET brand = regexp_extract(_corrupt_record, "(?<=brand': )(.*?)(?=, 'title|, 'description|, 'price|, 'salesRank|, 'imUrl|, 'related|, 'asin|, 'categories|})", 1);
UPDATE corrupt_products SET categories = regexp_extract(_corrupt_record, "(?<=categories': )(.*?)(?=, 'title|, 'description|, 'price|, 'salesRank|, 'imUrl|, 'related|, 'asin|, 'brand|})", 1);
UPDATE corrupt_products SET description = regexp_extract(_corrupt_record, "(?<=description': )(.*?)(?=, 'title|, 'categories|, 'price|, 'salesRank|, 'imUrl|, 'related|, 'asin|, 'brand|})", 1);
UPDATE corrupt_products SET imUrl = regexp_extract(_corrupt_record, "(?<=imUrl': )(.*?)(?=, 'title|, 'categories|, 'price|, 'salesRank|, 'description|, 'related|, 'asin|, 'brand|})", 1);
UPDATE corrupt_products SET price = regexp_extract(_corrupt_record, "(?<=price': )(.*?)(?=, 'title|, 'categories|, 'imUrl|, 'salesRank|, 'description|, 'related|, 'asin|, 'brand|})", 1);
UPDATE corrupt_products SET title = regexp_extract(_corrupt_record, "(?<=brand': )(.*?)(?=, 'title|, 'description|, 'price|, 'salesRank|, 'imUrl|, 'related|, 'asin|, 'categories|})", 1);

In [ ]:
%sql
UPDATE corrupt_products SET related.also_bought = regexp_extract(_corrupt_record, "(?<=also_bought': )(.*?)(?=, 'also_viewed|, 'bought_together|, 'buy_after_viewing|})", 1);
UPDATE corrupt_products SET related.also_viewed = regexp_extract(_corrupt_record, "(?<=also_viewed': )(.*?)(?=, 'also_bought|, 'bought_together|, 'buy_after_viewing|})", 1);
UPDATE corrupt_products SET related.bought_together = regexp_extract(_corrupt_record, "(?<=bought_together': )(.*?)(?=, 'also_bought|, 'also_viewed|, 'buy_after_viewing|})", 1);
UPDATE corrupt_products SET related.buy_after_viewing = regexp_extract(_corrupt_record, "(?<=buy_after_viewing': )(.*?)(?=, 'also_bought|, 'also_viewed|, 'bought_together|})", 1);

In [ ]:
%sql
UPDATE corrupt_products SET salesRank.Appliances = regexp_extract(_corrupt_record, "(?<=Appliances': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Arts, Crafts & Sewing` = regexp_extract(_corrupt_record, "(?<=Arts, Crafts & Sewing': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Automotive = regexp_extract(_corrupt_record, "(?<=Automotive': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Baby = regexp_extract(_corrupt_record, "(?<=Baby': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Beauty = regexp_extract(_corrupt_record, "(?<=Beauty': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Books = regexp_extract(_corrupt_record, "(?<=Books': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Camera &amp; Photo` = regexp_extract(_corrupt_record, "(?<=Camera__amp__Photo': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Cell Phones & Accessories` = regexp_extract(_corrupt_record, "(?<=Cell Phones & Accessories': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Clothing = regexp_extract(_corrupt_record, "(?<=Clothing': )(.*?)(?=, 'Appliances|, 'Arts__Crafts___Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera__amp__Photo|, 'Cell_Phones___Accessories|, 'Clothing|, 'Computers___Accessories|, 'Electronics|, 'Gift_Cards_Store|, 'Grocery___Gourmet_Food|, 'Health___Personal_Care|, 'Home__amp__Kitchen|, 'Home_Improvement|, 'Industrial___Scientific|, 'Jewelry|, 'Kitchen___Dining|, 'Magazines|, 'Movies___TV|, 'Music|, 'Musical_Instruments|, 'Office_Products|, 'Patio__Lawn___Garden|, 'Pet_Supplies|, 'Prime_Pantry|, 'Shoes|, 'Software|, 'Sports__amp__Outdoors|, 'Toys___Games|, 'Video_Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Computers & Accessories` = regexp_extract(_corrupt_record, "(?<=Computers & Accessories': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Electronics = regexp_extract(_corrupt_record, "(?<=Electronics': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Gift Cards Store` = regexp_extract(_corrupt_record, "(?<=Gift Cards Store': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Grocery & Gourmet Food` = regexp_extract(_corrupt_record, "(?<=Grocery & Gourmet Food': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Health & Personal Care` = regexp_extract(_corrupt_record, "(?<=Health & Personal Care': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Home &amp; Kitchen` = regexp_extract(_corrupt_record, "(?<=Home &amp; Kitchen': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Home Improvement` = regexp_extract(_corrupt_record, "(?<=Home Improvement': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Industrial & Scientific` = regexp_extract(_corrupt_record, "(?<=Industrial & Scientific': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Jewelry = regexp_extract(_corrupt_record, "(?<=Jewelry': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Kitchen & Dining` = regexp_extract(_corrupt_record, "(?<=Kitchen & Dining': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Magazines = regexp_extract(_corrupt_record, "(?<=Magazines': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Movies & TV` = regexp_extract(_corrupt_record, "(?<=Movies & TV': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Music = regexp_extract(_corrupt_record, "(?<=Music': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Musical Instruments` = regexp_extract(_corrupt_record, "(?<=Musical Instruments': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Office Products` = regexp_extract(_corrupt_record, "(?<=Office Products': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Patio, Lawn & Garden` = regexp_extract(_corrupt_record, "(?<=Patio, Lawn & Garden': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Pet Supplies` = regexp_extract(_corrupt_record, "(?<=Pet Supplies': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Prime Pantry` = regexp_extract(_corrupt_record, "(?<=Prime Pantry': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Shoes = regexp_extract(_corrupt_record, "(?<=Shoes': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Software = regexp_extract(_corrupt_record, "(?<=Software': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Sports &amp; Outdoors` = regexp_extract(_corrupt_record, "(?<=Sports &amp; Outdoors': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Toys & Games` = regexp_extract(_corrupt_record, "(?<=Toys & Games': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.`Video Games` = regexp_extract(_corrupt_record, "(?<=Video Games': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

UPDATE corrupt_products SET salesRank.Watches = regexp_extract(_corrupt_record, "(?<=Watches': )(.*?)(?=, 'Appliances|, 'Arts, Crafts & Sewing|, 'Automotive|, 'Baby|, 'Beauty|, 'Books|, 'Camera &amp; Photo|, 'Cell Phones & Accessories|, 'Clothing|, 'Computers & Accessories|, 'Electronics|, 'Gift Cards Store|, 'Grocery & Gourmet Food|, 'Health & Personal Care|, 'Home &amp; Kitchen|, 'Home Improvement|, 'Industrial & Scientific|, 'Jewelry|, 'Kitchen & Dining|, 'Magazines|, 'Movies & TV|, 'Music|, 'Musical Instruments|, 'Office Products|, 'Patio, Lawn & Garden|, 'Pet Supplies|, 'Prime Pantry|, 'Shoes|, 'Software|, 'Sports &amp; Outdoors|, 'Toys & Games|, 'Video Games|, 'Watches|})", 1);

In [ ]:
%sql
-- Insertar 331 filas limpiadas a nuestra tabla de metadata
INSERT INTO products5
  SELECT c.asin, c.brand, c.categories, c.description, c.imUrl, c.price, c.related, c.salesRank, c.title FROM corrupt_products AS c LEFT JOIN products5 AS p ON c.asin == p.asin WHERE p.asin IS NULL;

In [ ]:
%sql
SELECT COUNT(asin) FROM products5 WHERE asin IS NULL OR asin == ""; -- 0
SELECT COUNT(brand) FROM products5 WHERE brand IS NULL OR brand == ""; -- 68,421
SELECT COUNT(categories) FROM products5 WHERE categories IS NULL OR categories == "" OR categories == "[[""]]"; -- 7
SELECT COUNT(description) FROM products5 WHERE description IS NULL OR description == ""; -- 204,189
SELECT COUNT(imUrl) FROM products5 WHERE imUrl IS NULL OR imUrl == ""; -- 1
SELECT COUNT(price) FROM products5 WHERE price IS NULL; -- 0
SELECT COUNT(related) FROM products5 WHERE related IS NULL; -- 0
SELECT COUNT(salesRank) FROM products5 WHERE salesRank IS NULL; -- 0
SELECT COUNT(title) FROM products5 WHERE title IS NULL OR title == ""; -- 290

#### Todas las 9.430.088 fueron limpiadas. Pero aún así hay columnas con valores nulos o con strings vacías

In [ ]:
%sql
-- Cambiar nombre de nuestra tabla final y de sus columnas
ALTER TABLE products5 RENAME TO metadata_clean;

#### Cambiando el esquema de metadata que permita hacer el export a SQL Database Azure

In [ ]:
metadata = spark.sql("SELECT asin, title, categories, brand, description, price, related FROM metadata_clean")

In [ ]:
# Cambiando el nombre de asin a asinID
metadata = metadata.withColumnRenamed("asin", "asinID")

### Related Column

In [ ]:
from pyspark.sql.functions import col
metadata = metadata.withColumn("also_bought", col("related.also_bought")).withColumn("also_viewed", col("related.also_viewed")).withColumn("bought_together", col("related.bought_together"))\
            .withColumn("buy_after_viewing", col("related.buy_after_viewing")).drop("related")

In [ ]:
metadata.write.format("delta").saveAsTable("metadata_clean_v2")

In [ ]:
%sql
UPDATE metadata_clean_v2 SET also_bought = REPLACE(also_bought, "[", "");
UPDATE metadata_clean_v2 SET also_bought = REPLACE(also_bought, "]", "");

num_affected_rows
9430088


In [ ]:
%sql
UPDATE metadata_clean_v2 SET also_viewed = REPLACE(also_viewed, "[", "");
UPDATE metadata_clean_v2 SET also_viewed = REPLACE(also_viewed, "]", "");

num_affected_rows
9430088


In [ ]:
%sql
UPDATE metadata_clean_v2 SET bought_together = REPLACE(bought_together, "[", "");
UPDATE metadata_clean_v2 SET bought_together = REPLACE(bought_together, "]", "");

num_affected_rows
9430088


In [ ]:
%sql
UPDATE metadata_clean_v2 SET buy_after_viewing = REPLACE(buy_after_viewing, "[", "");
UPDATE metadata_clean_v2 SET buy_after_viewing = REPLACE(buy_after_viewing, "]", "");

num_affected_rows
9430088


### Categories Column

In [ ]:
%sql
UPDATE metadata_clean_v2 SET categories = REPLACE(categories, "[", "");
UPDATE metadata_clean_v2 SET categories = REPLACE(categories, "]", "");
UPDATE metadata_clean_v2 SET categories = REPLACE(categories, '"', "");

num_affected_rows
9430088


### Definición de Querys para la carga de datos delta

In [ ]:
product_table = spark.sql("SELECT DISTINCT(m.asinID), m.title, m.price, m.brand, m.categories FROM metadata_clean_v2 m JOIN reviews r ON m.asinID = r.asinID;")
product_table = set_nullable_false(spark, product_table, "asinID")

In [ ]:
# Petición de Machine Learning
reviews_2014_ml = spark.sql("SELECT r.reviewerID, r.asinID, r.reviewText, r.summary, r.overall, m.title, m.categories FROM reviews r JOIN metadata_clean_v2 m ON r.asinID = m.asinID WHERE YEAR(date) = 2014;")

In [ ]:
reviews_2014_ml.write.format("delta").saveAsTable("reviews_2014_ml")

In [ ]:
product_table_ml = spark.sql("SELECT asinID, description, also_bought, also_viewed, bought_together, buy_after_viewing FROM metadata_clean_v2")

# 3. Load

In [ ]:
from pyspark.sql import DataFrameWriter
def update_database(df, table: str, mode = "append"):
    # modes: overwrite, update
    '''Esta función recibe un dataframe de spark y exporta los datos a la base de datos de Azure SQL'''
    
    jdbcHostname = "database_server_name.database.windows.net"
    jdbcPort = "1433"
    jdbcDatabase = "database_name"
    properties = {"user": "username", "password": "password"}
    
    url = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname,jdbcPort,jdbcDatabase)
    
    df = DataFrameWriter(df)
    return df.jdbc(url = url, table = table, mode = mode, properties = properties)

In [ ]:
reviews_count = spark.sql("SELECT COUNT(reviewID), date FROM reviews WHERE YEAR(date) BETWEEN 2000 AND 2014 GROUP BY date")

In [ ]:
products_count = spark.sql("SELECT asinID, date FROM reviews WHERE YEAR(date) BETWEEN 2000 AND 2014")

In [ ]:
asinID_count = spark.sql("SELECT COUNT(DISTINCT(asinID)) Cuenta, YEAR(date) Anio FROM reviews WHERE YEAR(date) BETWEEN 2000 AND 2014 GROUP BY YEAR(date)")

In [ ]:
reviewers_count = spark.sql("SELECT COUNT(DISTINCT(reviewerID)) Cuenta, YEAR(date) Anio FROM reviews WHERE YEAR(date) BETWEEN 2000 AND 2014 GROUP BY YEAR(date)")

In [ ]:
update_database(reviews_count, table = "reviews_count", mode = "overwrite")

In [ ]:
update_database(asinID_count, table = "asinID_count", mode = "overwrite")

In [ ]:
update_database(reviewers_count, table = "reviewers_count", mode = "overwrite")

In [ ]:
# facts_reviews
update_database(reviews, table = "facts_reviews")

In [ ]:
# Users table
update_database(df_reviewerid_name, table = "users_table")

In [ ]:
# product table
update_database(product_table, table = "facts_product_table", mode = 'overwrite')

In [ ]:
# reviews
update_database(reviews_2014_ml, table = "reviews_2014_ml")

In [ ]:
# product table for ML
update_database(product_table_ml, table = "product_table_ml")